# Creating the Hawaii (HI) Dataset

#### From InsideAirbnb and POI-Factory, information about Airbnb listings, booking dates, and local attractions will be synthesized into a single dataset that can later be used for modeling and visualizations.  The data will be filtered, joined, and transformed primarily via the Pandas and GeoPandas python libraries.

## Requirements and imports 

In [1]:
! pip install -r requirements.txt

In [2]:
import pandas as pd
import geopandas as gpd

## Read in the Listings, Neighbourhoods, and Booking Calendar Data

In [3]:
HI_cal = pd.read_csv('./Hawaii/calendar.csv.gz', compression='gzip',)
HI_lis = pd.read_csv('./Hawaii/airbnb/hawaii_listings.csv')
HI_nei = pd.read_csv('./Hawaii/neighbourhoods.csv').fillna(0)

display(HI_cal.head(5))
display(HI_lis.head(5))
display(HI_nei.head(5))


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,5065,2021-07-09,f,$85.00,$85.00,2.0,700.0
1,87671,2021-07-09,f,$227.00,$227.00,7.0,1125.0
2,87671,2021-07-10,f,$227.00,$227.00,7.0,1125.0
3,87671,2021-07-11,f,$227.00,$227.00,7.0,1125.0
4,87671,2021-07-12,f,$227.00,$227.00,7.0,1125.0


,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,closeness_attractions
0,5065,https://www.airbnb.com/rooms/5065,2.020000e+13,7/9/2021,MAUKA BB,"Perfect for your vacation, Staycation or just ...",Neighbors here are friendly but are not really...,https://a0.muscache.com/pictures/36718112/1f0e...,7257,https://www.airbnb.com/users/show/7257,...,4.48,4.76,NaN,f,1,1,0,0,0.41,30
1,5269,https://www.airbnb.com/rooms/5269,2.020000e+13,7/9/2021,Upcountry Hospitality in the 'Auwai Suite,"The 'Auwai Suite is a lovely, self-contained a...","We are located on the ""sunny side"" of Waimea, ...",https://a0.muscache.com/pictures/5b52b72f-5a09...,7620,https://www.airbnb.com/users/show/7620,...,5.00,4.82,119-269-5808-01R,f,3,3,0,0,0.10,36
2,5387,https://www.airbnb.com/rooms/5387,2.020000e+13,7/9/2021,Hale Koa Studio & 1 Bedroom Units!!,This Wonderful Spacious Studio apt/flat is in ...,IN a Farm belt area with small commercial farm...,https://a0.muscache.com/pictures/1170713/dca6a...,7878,https://www.airbnb.com/users/show/7878,...,4.72,4.74,NaN,t,3,3,0,0,1.66,24
3,5389,https://www.airbnb.com/rooms/5389,2.020000e+13,7/9/2021,Keauhou Villa,It is less than 10 minute walk to the Keauhou ...,It is less than 10 minute walk to the Keauhou ...,https://a0.muscache.com/pictures/15520396/3b89...,7878,https://www.airbnb.com/users/show/7878,...,4.97,4.74,NaN,f,3,3,0,0,0.57,23
4,5390,https://www.airbnb.com/rooms/5390,2.020000e+13,7/9/2021,STAY AT PRINCE KUHIO!,"<b>The space</b><br />Prince Kuhio, Studio Uni...",NaN,https://a0.muscache.com/pictures/12955/af97ac5...,7887,https://www.airbnb.com/users/show/7887,...,4.90,4.67,42652226,f,1,1,0,0,2.07,92


,neighbourhood_group,neighbourhood
0,Hawaii,Hamakua
1,Hawaii,Kau
2,Hawaii,North Hilo
3,Hawaii,North Kohala
4,Hawaii,North Kona


## Clean the Dataset and Add Relevant Features 

In [4]:
def my_szn(date):
    mon = date.month
    day = date.day
    
    if mon < 3 or mon == 3 and day <=20:
        szn='Winter'
    elif mon < 6 or mon == 6 and day <=20:
        szn = 'Spring'
    elif mon < 9 or mon == 9 and day <=20:
        szn = 'Summer'
    elif mon < 12 or mon == 12 and day <=20:
        szn = 'Fall'
    else:
        szn='Winter'
    
    return szn


In [5]:
###############################################################################################################################
# Add in Seasonality markers such as Month, Season, Day of Week, etc. 
###############################################################################################################################
HI_cal['date'] = pd.to_datetime(HI_cal['date'], infer_datetime_format=True)
HI_cal['month'] = HI_cal['date'].dt.month
HI_cal['qtr'] = HI_cal['date'].dt.quarter
HI_cal['is_weekend'] = HI_cal['date'].dt.weekday >= 4
HI_cal['dayofweek'] = HI_cal['date'].dt.dayofweek
HI_cal['szn'] = HI_cal.apply(lambda x: my_szn(x['date']), axis=1 )

In [6]:
###############################################################################################################################
# Pivot the Calendar data so that each listing appears on only 1 row (primary key==listing_id)
###############################################################################################################################

HI_cal_piv = HI_cal.groupby(['listing_id','szn','is_weekend','available'])\
            .agg({'price':'count'}).reset_index()\
            .pivot(['listing_id'],['szn','is_weekend','available'],['price'])\
            .reset_index().fillna(0)

HI_cal_piv.columns = HI_cal_piv.columns.to_flat_index()
HI_cal_piv.columns = ['listing_id',
 'Fall_weekday_f', 'Fall_weekday_t', 'Fall_weekend_f', 'Fall_weekend_t',
 'Spring_weekday_f', 'Spring_weekday_t', 'Spring_weekend_f', 'Spring_weekend_t',
 'Summer_weekday_f', 'Summer_weekday_t', 'Summer_weekend_f', 'Summer_weekend_t',
 'Winter_weekday_f', 'Winter_weekday_t', 'Winter_weekend_f', 'Winter_weekend_t']

###############################################################################################################################
# Convert Binary indicator of Booked (cols ending in _t or _f) to a single to a vacancy metric based on Season and Day of Week
###############################################################################################################################

HI_cal_piv['Fall_weekday'] = HI_cal_piv['Fall_weekday_t']/(HI_cal_piv['Fall_weekday_t']+HI_cal_piv['Fall_weekday_f'])
HI_cal_piv['Fall_weekend'] = HI_cal_piv['Fall_weekend_t']/(HI_cal_piv['Fall_weekend_t']+HI_cal_piv['Fall_weekend_f'])

HI_cal_piv['Spring_weekday'] = HI_cal_piv['Spring_weekday_t']/(HI_cal_piv['Spring_weekday_t']+HI_cal_piv['Spring_weekday_f'])
HI_cal_piv['Spring_weekend'] = HI_cal_piv['Spring_weekend_t']/(HI_cal_piv['Spring_weekend_t']+HI_cal_piv['Spring_weekend_f'])

HI_cal_piv['Summer_weekday'] = HI_cal_piv['Summer_weekday_t']/(HI_cal_piv['Summer_weekday_t']+HI_cal_piv['Summer_weekday_f'])
HI_cal_piv['Summer_weekend'] = HI_cal_piv['Summer_weekend_t']/(HI_cal_piv['Summer_weekend_t']+HI_cal_piv['Summer_weekend_f'])

HI_cal_piv['Winter_weekday'] = HI_cal_piv['Winter_weekday_t']/(HI_cal_piv['Winter_weekday_t']+HI_cal_piv['Winter_weekday_f'])
HI_cal_piv['Winter_weekend'] = HI_cal_piv['Winter_weekend_t']/(HI_cal_piv['Winter_weekend_t']+HI_cal_piv['Winter_weekend_f'])

###############################################################################################################################
# Convert the Binary indicators to a vacancy metric for Season only
###############################################################################################################################

HI_cal_piv['Fall'] = HI_cal_piv[[ 'Fall_weekday_t', 'Fall_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[[ 'Fall_weekday_f', 'Fall_weekday_t', 'Fall_weekend_f', 'Fall_weekend_t',]].sum(axis=1))
HI_cal_piv['Spring'] = HI_cal_piv[[ 'Spring_weekday_t', 'Spring_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[[ 'Spring_weekday_f', 'Spring_weekday_t', 'Spring_weekend_f', 'Spring_weekend_t',]].sum(axis=1))
HI_cal_piv['Summer'] = HI_cal_piv[[ 'Summer_weekday_t', 'Summer_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[[ 'Summer_weekday_f', 'Summer_weekday_t', 'Summer_weekend_f', 'Summer_weekend_t',]].sum(axis=1))
HI_cal_piv['Winter'] = HI_cal_piv[[ 'Winter_weekday_t', 'Winter_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[[ 'Winter_weekday_f', 'Winter_weekday_t', 'Winter_weekend_f', 'Winter_weekend_t']].sum(axis=1))

###############################################################################################################################
# Convert the Binary indicators to a vacancy metric for Weekday vs Weekend only
###############################################################################################################################
HI_cal_piv['Weekday'] = HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',]].sum(axis=1)/(
                        HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',]].sum(axis=1)+
                        HI_cal_piv[['Fall_weekday_f','Spring_weekday_f','Summer_weekday_f','Winter_weekday_f',]].sum(axis=1))

HI_cal_piv['Weekend'] = HI_cal_piv[['Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[['Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)+
                        HI_cal_piv[['Fall_weekend_f','Spring_weekend_f','Summer_weekend_f','Winter_weekend_f',]].sum(axis=1))


###############################################################################################################################
# Create a grand total Vacancy metric
###############################################################################################################################
HI_cal_piv['NoVacancy']=HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',
                                    'Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)/(
                        HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',
                                    'Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)+
                        HI_cal_piv[['Fall_weekday_f','Spring_weekday_f','Summer_weekday_f','Winter_weekday_f',
                                    'Fall_weekend_f','Spring_weekend_f','Summer_weekend_f','Winter_weekend_f',]].sum(axis=1))

HI_cal_piv['Occupancy']=HI_cal_piv[['Fall_weekday_t','Spring_weekday_t','Summer_weekday_t','Winter_weekday_t',
                                    'Fall_weekend_t','Spring_weekend_t','Summer_weekend_t','Winter_weekend_t',]].sum(axis=1)


###############################################################################################################################
# Trim down columns
###############################################################################################################################
HI_cal_piv = HI_cal_piv[['listing_id','Occupancy','Fall_weekday','Fall_weekend','Spring_weekday','Spring_weekend',
                         'Summer_weekday','Summer_weekend','Winter_weekday','Winter_weekend',
                         'Fall','Spring','Summer','Winter',
                         'Weekday','Weekend','NoVacancy']]

HI_cal_piv

,listing_id,Occupancy,Fall_weekday,Fall_weekend,Spring_weekday,Spring_weekend,Summer_weekday,Summer_weekend,Winter_weekday,Winter_weekend,Fall,Spring,Summer,Winter,Weekday,Weekend,NoVacancy
0,5065,146.0,0.557692,0.538462,0.423913,0.211538,0.175,0.433333,NaN,NaN,0.549451,0.347222,0.353846,NaN,0.407609,0.392265,0.400000
1,5269,154.0,0.211538,0.205128,0.423913,0.615385,0.625,0.450000,0.428571,NaN,0.208791,0.493056,0.566667,0.428571,0.413386,0.441441,0.421918
2,5387,135.0,0.307692,0.282051,0.423913,0.346154,0.300,0.333333,0.476190,NaN,0.296703,0.395833,0.313433,0.476190,0.392713,0.322034,0.369863
3,5389,127.0,0.230769,0.230769,0.423913,0.384615,0.200,0.333333,0.448718,NaN,0.230769,0.409722,0.230769,0.448718,0.358779,0.320388,0.347945
4,5390,102.0,0.096154,0.076923,0.402439,0.173077,0.175,0.440000,0.430769,0.600000,0.087912,0.313433,0.276923,0.453333,0.305439,0.230159,0.279452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21803,50898372,157.0,0.403846,0.410256,0.423529,0.500000,0.400,0.453125,0.384615,0.428571,0.406593,0.452555,0.432692,0.393939,0.408867,0.456790,0.430137
21804,50898551,155.0,0.461538,0.384615,0.423913,0.480769,0.325,0.428571,0.439024,NaN,0.428571,0.444444,0.382022,0.439024,0.417778,0.435714,0.424658
21805,50900058,187.0,0.673077,0.794872,0.423913,0.442308,0.500,0.440000,0.430769,NaN,0.725275,0.430556,0.476923,0.430769,0.489960,0.560345,0.512329
21806,50900602,102.0,0.115385,0.153846,0.405405,0.153846,0.100,1.000000,0.413793,0.436364,0.131868,0.258427,0.162791,0.422535,0.282407,0.275168,0.279452


In [7]:
###############################################################################################################################
# Join the Calendar data to the Listing data and the Neighbourhood data
###############################################################################################################################
HI_df = HI_cal_piv.merge(HI_lis, left_on='listing_id', right_on='id', how='inner', suffixes=('','_lis'))
HI_df = HI_df.merge(HI_nei.drop('neighbourhood',axis=1), on=['neighbourhood_group'], how='inner', suffixes=('','_nei')).drop_duplicates()
print(HI_df.columns, HI_df.shape)
HI_df.head(5)


Index(['listing_id', 'Occupancy', 'Fall_weekday', 'Fall_weekend',
       'Spring_weekday', 'Spring_weekend', 'Summer_weekday', 'Summer_weekend',
       'Winter_weekday', 'Winter_weekend', 'Fall', 'Spring', 'Summer',
       'Winter', 'Weekday', 'Weekend', 'NoVacancy', 'id', 'listing_url',
       'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'bed

,listing_id,Occupancy,Fall_weekday,Fall_weekend,Spring_weekday,Spring_weekend,Summer_weekday,Summer_weekend,Winter_weekday,Winter_weekend,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,closeness_attractions
0,5065,146.0,0.557692,0.538462,0.423913,0.211538,0.175,0.433333,NaN,NaN,...,4.48,4.76,NaN,f,1,1,0,0,0.41,30
9,5269,154.0,0.211538,0.205128,0.423913,0.615385,0.625,0.450000,0.428571,NaN,...,5.00,4.82,119-269-5808-01R,f,3,3,0,0,0.10,36
18,5387,135.0,0.307692,0.282051,0.423913,0.346154,0.300,0.333333,0.476190,NaN,...,4.72,4.74,NaN,t,3,3,0,0,1.66,24
27,5389,127.0,0.230769,0.230769,0.423913,0.384615,0.200,0.333333,0.448718,NaN,...,4.97,4.74,NaN,f,3,3,0,0,0.57,23
36,8833,120.0,0.192308,0.102564,0.437500,0.307692,0.300,0.285714,0.445783,0.333333,...,4.74,4.58,W95781870,f,1,1,0,0,2.54,23


## Read in the POI Factory Data 

In [8]:
HI_poi_rest = pd.read_csv("./Hawaii/poi_factory/AK_Air_Rewards_Restraunts_HI.csv",header=None)[[0,1,2]]
HI_poi_mili = pd.read_csv("./Hawaii/poi_factory/Hawaii historic military sites.csv",header=None)[[0,1,2]]
HI_poi_beaK = pd.read_csv("./Hawaii/poi_factory/Kauai Beaches.csv",header=None)[[0,1,2]]
HI_poi_sigK = pd.read_csv("./Hawaii/poi_factory/Kauai Sights.csv",header=None)[[0,1,2]]
HI_poi_shoK = pd.read_csv("./Hawaii/poi_factory/Kauai Shopping.csv",header=None)[[0,1,2]]
HI_poi_beaO = pd.read_csv("./Hawaii/poi_factory/Oahu Beaches.csv",header=None)[[0,1,2]]
HI_poi_sigO = pd.read_csv("./Hawaii/poi_factory/Oahu Sights.csv",header=None)[[0,1,2]]
HI_poi_shoO = pd.read_csv("./Hawaii/poi_factory/Oahu Shopping.csv",header=None)[[0,1,2]]
HI_poi_snor = pd.read_csv("./Hawaii/poi_factory/Snorkel Bob.csv",header=None)[[0,1,2]]
HI_poi_tour = pd.read_csv("./Hawaii/poi_factory/USAPOI Tourist Guide for _Hawaii.csv",header=None)[[0,1,2]]


HI_poi = pd.concat([HI_poi_rest, HI_poi_mili, HI_poi_beaK, HI_poi_sigK, HI_poi_shoK,
                    HI_poi_beaO, HI_poi_sigO, HI_poi_shoO, HI_poi_snor, HI_poi_tour ])

HI_poi.columns = ['POI_LNG','POI_LAT','POI_name']
HI_poi

,POI_LNG,POI_LAT,POI_name
0,-159.320679,22.061848,La Playita Azul
1,-157.820877,21.273705,Matteo s Italian
2,-157.850525,21.292034,Mocha Java Cafe
3,-156.677338,20.872435,Mulligan s At The Wharf
4,-156.437607,20.674744,Mulligan s On The
...,...,...,...
39,-155.881076,19.418295,South Kona Fruit Stand
40,-155.995168,19.643350,Kona Tacos
41,-159.670837,21.958202,Daniel Thiebaut
42,-155.070245,19.725163,Nihon Restaurant


## Convert Pandas to GeoPandas, Project the Data to the Correct EPSG Codes, Join Datasets

In [9]:
###############################################################################################################################
# The EPSG code used for the Greater Hawaii area is EPSG:3832 which is the PDC Mercator.  For more info, see blog post.
###############################################################################################################################
 
HI_poi_g = gpd.GeoDataFrame(HI_poi, geometry=gpd.points_from_xy(HI_poi.POI_LNG, HI_poi.POI_LAT), crs="EPSG:4326")
HI_poi_g = HI_poi_g.to_crs("EPSG:3832")
HI_poi_g.crs

<Projected CRS: EPSG:3832>
Name: WGS 84 / PDC Mercator
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Pacific Ocean - American Samoa, Antarctica, Australia, Brunei Darussalam, Cambodia, Canada, Chile, China, China - Hong Kong, China - Macao, Cook Islands, Ecuador, Fiji, French Polynesia, Guam, Indonesia, Japan, Kiribati, Democratic People's Republic of Korea (North Korea), Republic of Korea (South Korea), Malaysia, Marshall Islands, Federated States of Micronesia, Nauru, New Caledonia, New Zealand, Niue, Norfolk Island, Northern Mariana Islands, Palau, Panama, Papua New Guinea (PNG), Peru, Philippines, Pitcairn, Russian Federation, Samoa, Singapore, Solomon Islands, Taiwan, Thailand, Tokelau, Tonga, Tuvalu, United States (USA), United States Minor Outlying Islands, Vanuatu, Venezuela, Vietnam, Wallis and Futuna.
- bounds: (98.69, -60.0, -68.0, 66.67)
Coordinate Operation:
- name: Pacific Disaster Center Mercator
- method: Mercator (v

In [10]:
###############################################################################################################################
# In order to join in the next cell, both EPSG projections must be the same.
###############################################################################################################################
HI_df_g = gpd.GeoDataFrame(HI_df, geometry=gpd.points_from_xy(HI_df.longitude, HI_df.latitude), crs="EPSG:4326")
HI_df_g = HI_df_g.to_crs("EPSG:3832")
HI_df_g.crs

<Projected CRS: EPSG:3832>
Name: WGS 84 / PDC Mercator
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Pacific Ocean - American Samoa, Antarctica, Australia, Brunei Darussalam, Cambodia, Canada, Chile, China, China - Hong Kong, China - Macao, Cook Islands, Ecuador, Fiji, French Polynesia, Guam, Indonesia, Japan, Kiribati, Democratic People's Republic of Korea (North Korea), Republic of Korea (South Korea), Malaysia, Marshall Islands, Federated States of Micronesia, Nauru, New Caledonia, New Zealand, Niue, Norfolk Island, Northern Mariana Islands, Palau, Panama, Papua New Guinea (PNG), Peru, Philippines, Pitcairn, Russian Federation, Samoa, Singapore, Solomon Islands, Taiwan, Thailand, Tokelau, Tonga, Tuvalu, United States (USA), United States Minor Outlying Islands, Vanuatu, Venezuela, Vietnam, Wallis and Futuna.
- bounds: (98.69, -60.0, -68.0, 66.67)
Coordinate Operation:
- name: Pacific Disaster Center Mercator
- method: Mercator (v

In [11]:
###############################################################################################################################
# Add a 1600 meter Buffer around each Listing point (1600m = .99 miles). 
# Do a Spatial Within Join to match any POI point within the 1 mile buffer around each Listing point.
###############################################################################################################################
 
HI_df_g['geometry'] = HI_df_g['geometry'].buffer(1600)
HI_f = gpd.sjoin(HI_poi_g, HI_df_g, how='right', op='within').sort_values(by=['id','POI_name'])
HI_f


,index_left,POI_LNG,POI_LAT,POI_name,listing_id,Occupancy,Fall_weekday,Fall_weekend,Spring_weekday,Spring_weekend,...,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,closeness_attractions,geometry
0,NaN,NaN,NaN,NaN,5065,146.0,0.557692,0.538462,0.423913,0.211538,...,4.76,NaN,f,1,1,0,0,0.41,30,"POLYGON ((6076016.295 2263448.104, 6076008.591..."
9,NaN,NaN,NaN,NaN,5269,154.0,0.211538,0.205128,0.423913,0.615385,...,4.82,119-269-5808-01R,f,3,3,0,0,0.10,36,"POLYGON ((6046025.711 2261650.637, 6046018.007..."
18,39.0,-155.881076,19.418295,South Kona Fruit Stand,5387,135.0,0.307692,0.282051,0.423913,0.346154,...,4.74,NaN,t,3,3,0,0,1.66,24,"POLYGON ((6026134.031 2191515.392, 6026126.327..."
27,3.0,-155.958868,19.561911,Ali'i Mountain Course at Kona Country Club,5389,127.0,0.230769,0.230769,0.423913,0.384615,...,4.74,NaN,f,3,3,0,0,0.57,23,"POLYGON ((6017083.757 2207389.909, 6017076.052..."
43092,3.0,-159.476190,21.881470,Beach House (Kolopa) Beach,5390,102.0,0.096154,0.076923,0.402439,0.173077,...,4.67,42652226,f,1,1,0,0,2.07,92,"POLYGON ((5626188.704 2481387.166, 5626181.000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129012,NaN,NaN,NaN,NaN,50898372,157.0,0.403846,0.410256,0.423529,0.500000,...,NaN,GE-140-773-5808-01,t,137,137,0,0,NaN,60,"POLYGON ((5936492.152 2371022.344, 5936484.448..."
129021,NaN,NaN,NaN,NaN,50898551,155.0,0.461538,0.384615,0.423913,0.480769,...,NaN,NaN,t,137,137,0,0,NaN,60,"POLYGON ((5936348.161 2370941.880, 5936340.456..."
129030,NaN,NaN,NaN,NaN,50900058,187.0,0.673077,0.794872,0.423913,0.442308,...,NaN,NaN,t,137,137,0,0,NaN,60,"POLYGON ((5956452.416 2352368.637, 5956444.712..."
129039,NaN,NaN,NaN,NaN,50900602,102.0,0.115385,0.153846,0.405405,0.153846,...,NaN,175-718-1952-01,t,261,257,4,0,NaN,60,"POLYGON ((5935975.374 2369868.340, 5935967.669..."


In [12]:
###############################################################################################################################
# Some code brought in from outside, sourced by nkmartin
###############################################################################################################################
###############################################################################################################################
# Haversine Distance calculations are very computationally expesive and usually require a full cross join
# This project uses it only for validation purposes of the spatial join used above
###############################################################################################################################

import math
def caluclate_distance_haversine(store_a_longitude, store_a_latitude, store_b_longitude, store_b_latitude):

    # Calculate Latitude and Longitude Difference
    latitude_difference = (store_b_latitude - store_a_latitude) * math.pi / 180.0
    longitude_difference = (store_b_longitude - store_a_longitude) * math.pi / 180.0
  
    # Convert decimal degrees to radians 
    store_a_latitude = (store_a_latitude) * math.pi / 180.0
    store_b_latitude = (store_b_latitude) * math.pi / 180.0
  
    # Compute Haversine Formula
    a = (pow(math.sin(latitude_difference / 2), 2) + 
         pow(math.sin(longitude_difference / 2), 2) * 
             math.cos(store_a_latitude) * math.cos(store_b_latitude)); 
    rad = 6371.0
    c = 2 * math.asin(math.sqrt(a)) 
    return rad * c 

## Verify with a simple Visualization 

In [13]:
###############################################################################################################################
# Take a small subset of the data for graphing purposes
###############################################################################################################################
fol_tmp = HI_f[['POI_LAT','POI_LNG','POI_name','id','latitude','longitude']]#.head(1500)
fol_tmp = fol_tmp[fol_tmp['id'].isin([18140683,48230317])]
fol_tmp['hav_dist_km'] = fol_tmp.apply(lambda x: 
                                    caluclate_distance_haversine(float(x.POI_LAT), float(x.POI_LNG), float(x.latitude),float(x.longitude))
                                    , axis=1)
fol_tmp['hav_dist_mi'] = fol_tmp['hav_dist_km'] * 0.621371
fol_tmp = fol_tmp[(fol_tmp['hav_dist_mi']<=1)]
fol_tmp.sort_values('id').dropna()

,POI_LAT,POI_LNG,POI_name,id,latitude,longitude,hav_dist_km,hav_dist_mi
139816,21.289858,-157.839676,Aaron s Atop The Ala Moana,18140683,21.28575,-157.83153,0.999710,0.621191
139816,21.279140,-157.830930,Waikiki Beach Walk,18140683,21.28575,-157.83153,0.683927,0.424972
139816,21.276300,-157.827930,Waikiki Beach,18140683,21.28575,-157.83153,1.052222,0.653820
139816,21.289738,-157.832703,Tudo De Bom,18140683,21.28575,-157.83153,0.430882,0.267738
139816,21.278370,-157.831803,SpaHalekulani at Halekulani,18140683,21.28575,-157.83153,0.760564,0.472592
139816,21.284670,-157.835815,Sergio s Italian,18140683,21.28575,-157.83153,0.489278,0.304023
139816,21.280245,-157.826935,Seaside Bar &,18140683,21.28575,-157.83153,0.763152,0.474201
139816,21.285191,-157.838654,Sarento s Top Of,18140683,21.28575,-157.83153,0.794241,0.493519
139816,21.278780,-157.828760,Royal Hawaiian Shopping Center,18140683,21.28575,-157.83153,0.781029,0.485309
139816,21.274715,-157.824066,Popeye s Chicken,18140683,21.28575,-157.83153,1.407129,0.874349


In [14]:
###############################################################################################################################
# Plot first the POI points, then the 1 mile buffer, then the airbnb listing
###############################################################################################################################
import folium
mapit=None
mapit = folium.Map(location=[21.28575,-157.83153], zoom_start=14)

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(mapit)

small = 250 

print("Blue area is the 1 mile radius around each Listing")
fol_tmp.head(small).drop_duplicates().apply(lambda row:folium.Circle(location=[row["latitude"], row["longitude"]], 
                                              radius=1600, fill_color='blue', fill_opacity=0.01)
                                             .add_to(mapit), axis=1)
print("Red points are the Listings")
fol_tmp.head(small).drop_duplicates().apply(lambda row:folium.Circle(location=[row["latitude"], row["longitude"]], 
                                              radius=20, fill_color='red', color='red', fill_opacity=1)
                                             .add_to(mapit), axis=1)
print("Orange points are the POI")
fol_tmp.head(small).drop_duplicates().apply(lambda row:folium.Circle(location=[row["POI_LAT"], row["POI_LNG"]], 
                                              radius=10, fill_color='orange', color='orange',fill_opacity=1)
                                             .add_to(mapit) if row["POI_LAT"] else 0, axis=1)

Blue area is the 1 mile radius around each Listing
Red points are the Listings
Orange points are the POI


139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle object at 0x7fa14...
139816    <folium.vector_layers.Circle o

In [15]:
mapit

## Run a GB to Count the POI by Listing

In [16]:
HI_f.groupby(['listing_id','neighbourhood_group','room_type','Occupancy','NoVacancy'])\
        .agg({'POI_name':pd.Series.nunique}).reset_index()\
        .sort_values(by=['POI_name'], ascending=False).rename(columns={'POI_name':'Num_POI_within_1mile'})

,listing_id,neighbourhood_group,room_type,Occupancy,NoVacancy,Num_POI_within_1mile
18909,48230317,Honolulu,Private room,209.0,0.572603,28
5099,18140683,Honolulu,Entire home/apt,170.0,0.465753,28
17651,47030949,Honolulu,Entire home/apt,190.0,0.520548,28
17022,46196780,Honolulu,Entire home/apt,168.0,0.460274,28
375,873736,Honolulu,Entire home/apt,197.0,0.539726,28
...,...,...,...,...,...,...
9981,30558612,Hawaii,Entire home/apt,229.0,0.627397,0
9983,30562975,Honolulu,Private room,136.0,0.372603,0
9985,30564082,Maui,Entire home/apt,127.0,0.347945,0
9986,30565415,Hawaii,Entire home/apt,238.0,0.652055,0


## Save as Tall table with each row representing a POI-Listing Match

In [17]:
HI_f.to_csv("./Hawaii_v1.csv.gz", compression='gzip')